In [ ]:
import os
import plotly.express as px
import pandas as pd
import numpy as np
import json

from utils import *
from train import *
from eval import *
import wandb
wandb.login()

import warnings
warnings.filterwarnings('ignore')

# Set seed
np.random.seed(42)


# Set working directory
os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"
dir_path = os.path.join(os.getcwd(), 'data', 'clean_data')
model_dir = os.path.join(os.getcwd(), 'models')



In [ ]:
def train_eval_light():

    wandb.init(project="WattCast_tuning")
    wandb.config.update(config_run)
    config = wandb.config

    print("Getting data...")

    pipeline, ts_train_piped, ts_val_piped, ts_test_piped, ts_train_weather_piped, ts_val_weather_piped, ts_test_weather_piped, trg_train_inversed, trg_val_inversed, trg_test_inversed = data_pipeline(config)

    print("Getting model instance...")
    model_instance = get_model(config)
    model_instance, _ = train_models([model_instance], ts_train_piped, ts_train_weather_piped, ts_val_piped, ts_val_weather_piped)

    print("Evaluating model...")
    predictions, score = predict_testset(model_instance[0], 
                                  ts_test_piped[config.longest_ts_test_idx], 
                                  ts_test_weather_piped[config.longest_ts_test_idx],
                                  config.n_lags, config.n_ahead, config.eval_stride, pipeline,
                                  )


    print("Plotting predictions...")
    df_compare = pd.concat([trg_test_inversed.pd_dataframe(), predictions], axis=1).dropna()
    df_compare.columns = ['target', 'prediction']
    fig = px.line(df_compare, title='Predictions vs. Test Set')

    wandb.log({'eval_loss': score})
    wandb.log({'predictions': fig})
    wandb.finish()



In [ ]:

# See what keys are in the h5py data file
locations_dict, resolutions_dict = get_hdf_keys(dir_path)

print("Locations: ", locations_dict)
print("Resolutions: ", resolutions_dict)

for scale, locations in locations_dict.items():
    if scale == '5_building.h5':
        for location in locations:
            df_train = pd.read_hdf(os.path.join(dir_path, scale), key=f'{location}/60min/train_target')
            df_val = pd.read_hdf(os.path.join(dir_path, scale), key=f'{location}/60min/val_target')
            df_test = pd.read_hdf(os.path.join(dir_path, scale), key=f'{location}/60min/test_target')
            fig = px.line(df_train, title=f'{scale}: {location}')
            fig.add_trace(px.line(df_val, title='Validation Set').data[0])
            fig.add_trace(px.line(df_test, title='Test Set').data[0])
            fig.show()



### Running

In [ ]:
# run parameters

sweeps = 20

scale_location_pairs = (
    # ('1_county', 'Sacramento'),
    # ('1_county', 'New_York'),
    #('2_town', 'town_0'),
    # ('2_town', 'town_1'),
    # ('2_town', 'town_2'),
    # ('3_village', 'village_1'),
    # ('3_village', 'village_2'),
    #('3_village', 'village_0'),
    # ('4_neighborhood', 'neighborhood_0'),
    # ('4_neighborhood', 'neighborhood_1'),
    # ('4_neighborhood', 'neighborhood_2'),
    # ('5_building', 'building_0'),
    # ('5_building', 'building_1'),
    ('5_building', 'building_2'),
      )



models = [
        'rf',
         'xgb', 
        # 'gru', 
        'lgbm',  
        # 'nbeats',
        #'tft'
        ]


for scale, location in scale_location_pairs:

    for model in models:
        # placeholder initialization of config file (will be updated in train_eval_light())
        config_run = {
            'spatial_scale': scale,
            'temp_resolution': 60,
            'location': location,
            'model': model,
            'horizon_in_hours': 24,
            'lookback_in_hours': 24,
            'boxcox': True,
            'liklihood': None,
            'weather': True,
            'holiday': True,
            'datetime_encodings': False,
        }
        
        with open(f'sweep_configurations/config_sweep_{model}.json', 'r') as fp:
            sweep_config = json.load(fp)                  

        sweep_config['name'] = model + 'sweep' + config_run['spatial_scale'] + '_' + config_run['location'] + '_' + str(config_run['temp_resolution'])

        sweep_id = wandb.sweep(sweep_config, project="WattCast_tuning")
        wandb.agent(sweep_id, train_eval_light, count=sweeps)


### Degbugging

In [ ]:
model = 'gru'
scale = '5_building'
location = 'building_2'


config_run = {
    'spatial_scale': scale,
    'temp_resolution': 60,
    'location': location,
    'model': model,
    'horizon_in_hours': 24,
    'lookback_in_hours': 24,
    'boxcox': True,
    'liklihood': None,
    'weather': True,
    'holiday': True,
    'datetime_encodings': False,
}

with open(f'sweep_configurations/config_sweep_{model}.json', 'r') as fp:
    sweep_config = json.load(fp)   


config = Config().from_dict(config_run)


In [ ]:
pipeline, ts_train_piped, ts_val_piped, ts_test_piped, ts_train_weather_piped, ts_val_weather_piped, ts_test_weather_piped, trg_train_inversed, trg_val_inversed, trg_test_inversed = data_pipeline(config)

In [ ]:
model_instance = get_model(config)

In [ ]:
type(model_instance)

In [ ]:
model_instance, _ = train_models([model_instance], ts_train_piped, ts_train_weather_piped, 
                                    ts_val_piped, ts_val_weather_piped
                                    )


In [ ]:
predictions, score = predict_testset(model_instance[0], 
                                ts_test_piped[config.longest_ts_test_idx], 
                                ts_test_weather_piped[config.longest_ts_test_idx],
                                config.n_lags, config.n_ahead, config.eval_stride, pipeline,
                                )

In [ ]:
px.line(predictions)